In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import datetime as dt
import pickle
import seaborn as sns
import numpy as np

%matplotlib inline

I plan to spend the bulk of my effort looking at the marathoners (full, not half) who participated in all 4 races. I may also spend time (time permitting) on racers who ran fewer races. 
first I want to look at racers who ran 16 and 17.  I wonder if I can predict who will race in 17 by seeing what their times are?
then i'll look at both 16 vs 18 to see if I can figure out about those who skipped the middle year, and do the same as above and predict if they will run in the 19 race
Then I want to look at 16, 17, and 18 to see if my predictions about the double racers coming back for third is correct, and to what extent. there are 48 runners who did this, I know from previous research that is the correct number. 
If I can determine a pattern, I want to look at those who raced all 4 races and see if I can any pattern there.

In [2]:
M2016 = pd.read_pickle("data/Marathon16.pkl")

In [3]:
M2016.shape

(3842, 4)

In [4]:
M2017 = pd.read_pickle("data/Marathon17.pkl")

In [5]:
M2017.shape

(3651, 4)

In [129]:
M2018 = pd.read_pickle("data/Marathon18.pkl")

In [130]:
M2019 = pd.read_pickle("data/Marathon19.pkl")

In [131]:
M1617 = pd.merge(M2016, M2017, on = ['Name'], how="inner")

In [108]:
M1617.shape

(368, 7)

In [135]:
#Let's rename some columns:
M1617 = M1617.rename(columns = {"Overall_x":"Overall_16", "Bib_x":"Bib_16", "Time_x":"Time_16", "Overall_y":"Overall_17", "Bib_y":"Bib_17", "Time_y":"Time_17"})

In [136]:
M1617.head()

,Overall_16,Bib_16,Name,Time_16,Overall_17,Bib_17,Time_17
0,1,1,Scott Wietecha,02:25:42,1,1,02:40:25
1,36,1070,Edgar Garza,03:13:18,6,1135,03:06:20
2,37,3026,Jonathan Hill,03:13:42,40,2051,03:28:27
3,54,1134,James Nabholz,03:17:41,99999,1038,00:00:00
4,68,4085,Tien Le,03:22:08,57,3128,03:32:19


In [111]:
M1718 = pd.merge(M2017, M2018, on = ['Name'], how="inner")

In [112]:
M1718.shape

(245, 7)

In [137]:
M161718 = pd.merge(M1617, M2018, on = ["Name"], how="inner")

In [155]:
M161718.shape

(103, 10)

Interesting... Slightly different number Than I got previously. I'll have to explore that a bit. Maybe I have some runners repeating names? I remember there were some weird names where the same name ran 5 or 6 races, impossibly. Obviously it was 2 runners with the same name. I'll try to weed those out a bit. 

In [138]:
M16to19 = pd.merge(M161718, M2019, on = ["Name"], how="inner")

Ok, now I am going to rename all those weird columns so I know what each refers to. 

In [139]:
M16to19 = M16to19.rename(columns = {"Overall_x":"Overall_118", "Bib_x":"Bib_18", "Time_x":"Time_18", "Overall_y":"Overall_19", "Bib_y":"Bib_19", "Time_y":"Time_19"})

In [182]:
M16to19.shape
#wacky that there are 65, not the 48 I found before. I'll make a csv file and look in the text editor to see if I have weird data
#and i did, there are a few runners with times of 0 for at least 1 race. I'll get rid of them

(65, 13)

In [141]:
all4x = M16to19[M16to19['Time_16'] > '02:25:42']

In [142]:
all4y= all4x[all4x['Time_17'] > '02:40:25']

In [143]:
all4z = all4y[all4y['Time_18'] > '02:28:16']

In [144]:
all4 = all4z[all4z['Time_19'] > '02:34:59']

#### 

In [181]:
all4.shape

(50, 13)

In [145]:
all4.head(55)
#ok still have 50, not 48. 

,Overall_16,Bib_16,Name,Time_16,Overall_17,Bib_17,Time_17,Overall_118,Bib_18,Time_18,Overall_19,Bib_19,Time_19
1,36,1070,Edgar Garza,03:13:18,6,1135,03:06:20,20,30048,03:05:40,12,30038,02:56:40
3,68,4085,Tien Le,03:22:08,57,3128,03:32:19,123,30060,03:34:43,192,30069,03:41:15
4,91,2005,Edwin Reyes,03:25:57,122,4128,03:44:12,120,30098,03:33:52,112,30109,03:30:56
7,145,2124,Lauren Terrell,03:31:51,62,2123,03:33:02,275,33455,03:53:29,99,33168,03:28:54
8,179,405,Dan Albert,03:36:19,2465,429,06:38:15,707,517,04:25:38,754,101,04:26:51
9,186,7143,Paul Bradbury,03:37:03,402,6115,04:13:34,411,30560,04:03:45,498,30549,04:07:27
10,190,1147,Dwayne Novak,03:37:47,303,1000,04:04:21,152,30141,03:37:34,204,30165,03:42:50
11,227,4081,Jonathan Lister,03:41:11,116,4110,03:43:43,77,30236,03:26:47,61,30203,03:21:31
12,265,5008,Erik Stephan,03:44:19,188,3057,03:51:58,156,30680,03:39:03,240,30464,03:46:28
13,282,4039,William Moore,03:45:20,273,5033,03:59:45,158,30336,03:39:19,194,30364,03:41:25


In [123]:
all4.Name.value_counts()
#aw, the infamous Dick Brooks. Right.

Richard Brooks      2
Jonathan Lister     1
Brett Smith         1
Dan Albert          1
Luke Tuttle         1
Ken Ashby           1
Alan Pettit         1
Ronald Woodall      1
Edgar Garza         1
Naomi Ryan          1
Jane Tao            1
Amanda Staggs       1
Edwin Reyes         1
Terry Dietzler      1
Jasser Mahmoud      1
Tien Le             1
Guido Hussels       1
Erik Stephan        1
Larry Epps          1
James Evans         1
Daniel Albert       1
Chad Koyama         1
Larry Phillips      1
Brooks Woloshyn     1
Frank Sames         1
Eddie Godwin        1
Bernard Pickney     1
Lauren Terrell      1
Dwayne Novak        1
Dallas Smith        1
Kelsey Otten        1
Melissa Boothe      1
Fernando Marquez    1
Carl Dunphy         1
Vincent Berry       1
Nghiem Le           1
Marla Albert        1
Paul Bradbury       1
Debra Shone         1
Kenneth Fiorilli    1
Deanna Alexander    1
Justin Milam        1
William Moore       1
Dean Bowden         1
Lauren Johnston     1
Angela Whi

In [147]:
all4D = all4[all4['Time_18'] != '06:00:39']
#I picked one of the times. I honestly don't know which was the bad one. HOpe he's not offended I chose his worst time to keep

In [148]:
all4D.shape 

(49, 13)

Ok, I found the one name that appears in this list but not my former list in excel. It is "Deanna Alexander". 
In excel, she appears in only 3 of the 4 races. Not sure why I did't pick her up in all 4.
It appears that in 2017, she didn't race... At least according to my Excel spreadsheet. At this point I am not sure what happened, exactly.


In [191]:
threetofour = pd.merge(M161718, M2019, on = ['Name'], how="left")

In [192]:
threetofour.shape

(106, 14)

In [156]:
threetofour.to_csv("data/3to4.csv")

In [157]:
all4D.to_csv("data/all4.csv")

In [158]:
M2016['Year'] = '2016'

In [193]:
M2016.sort_values("Name", inplace = True) 

In [196]:
M2016.drop_duplicates(subset ="Name", 
                     keep = False, inplace = True) 

In [197]:
M2016.shape

(3798, 5)

In [160]:
M2017['Year'] = '2017'

In [198]:
M2017.sort_values("Name", inplace = True) 

In [199]:
M2017.drop_duplicates(subset ="Name", 
                     keep = False, inplace = True) 

In [161]:
M2018['Year'] = '2018'

In [200]:
M2018.sort_values("Name", inplace = True) 

In [201]:
M2018.drop_duplicates(subset ="Name", 
                     keep = False, inplace = True) 

In [162]:
M2019['Year'] = '2019'

In [202]:
M2019.sort_values("Name", inplace = True) 

In [203]:
M2019.drop_duplicates(subset ="Name", 
                     keep = False, inplace = True)

In [204]:
M16M17 = pd.merge(M2016, M2017, on = ['Overall', 'Bib', 'Name', 'Time', 'Year'], how="outer")

In [205]:
M16M17M18 = pd.merge(M16M17, M2018, on = ['Overall', 'Bib', 'Name', 'Time', 'Year'], how="outer")

In [206]:
M16_M19 = pd.merge(M16M17M18, M2019, on = ['Overall', 'Bib', 'Name', 'Time', 'Year'], how="outer")

In [207]:
M16_M19.shape

(12280, 5)

In [208]:
M16_M19.to_csv("data/M16_M19.csv")

In [209]:
M16_M19.to_pickle("data/M16_M19.pkl")